# Deploying a Slice with a Prometheus Node Exporter

This notebook shows how to deploy [Prometheus](https://prometheus.io/) node exporters in your slice so that you can monitor them with [Grafana](https://grafana.com/).  This notebook creates the example slice that can be monitored by the Grafana server deployed in the [companion example notebook](grafana-prometheus-servers.ipynb).





## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment

Create a FABRIC experiment that supports Docker containers. The example using FABRIC's Docker Rocky 8 images (`docker_rocky_8`).  

All nodes in the slice will require network connectivity to the Grafana server deployed in the [companion notebook](grafana-prometheus-servers.ipynb).  The example uses the `add_fabnet()` function to add a FABNetv4 network with default configuration. You can use any network type that works with your experiment. 

Prometheus Node Exporter is started in a Docker container using the images supported by the vendor:

- Prometheus Node Exporter Docker image [prom/node-exporter](https://hub.docker.com/r/prom/node-exporter)

After the slice becomes active, the node exporters will be up and you can proceed to the [companion notebook](grafana-prometheus-servers.ipynb) to configure the Grafana/Prometheus servers.




In [ ]:
slice_name = 'MySlice'
[site1,site2] = fablib.get_random_sites(count=2)

print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, site=site1, image='docker_rocky_8')
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
node1.add_fabnet()
node1.add_post_boot_upload_directory('node_tools','.')
node1.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
node1.add_post_boot_upload_directory("./node_exporter", ".")
node1.add_post_boot_execute(f'cd node_exporter; docker compose up -d')


# Node2
node2 = slice.add_node(name=node2_name, site=site2)
node2.add_fabnet()
node2.add_post_boot_upload_directory('node_tools','.')
node2.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
node2.add_post_boot_upload_directory("./node_exporter", ".")
node2.add_post_boot_execute(f'cd node_exporter; docker compose up -d')

#Submit Slice Request
slice.submit();

## Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=f'FABNET_IPv4_{node2.get_site()}').get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    



## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()